In [1]:
%%capture --no-stderr
%pip install --upgrade --quiet langgraph langchain-community beautifulsoup4

In [2]:
# from langchain_community.document_loaders import WebBaseLoader

# urls = [
#     "https://lilianweng.github.io/posts/2024-11-28-reward-hacking/",
#     "https://lilianweng.github.io/posts/2024-07-07-hallucination/",
#     "https://lilianweng.github.io/posts/2024-04-12-diffusion-video/",
# ]

# docs = [WebBaseLoader(url).load() for url in urls]

In [3]:
from langchain_community.document_loaders import WebBaseLoader

urls = [
    "https://www.thehindubusinessline.com/topic/credit-ratings/",
]

docs = [WebBaseLoader(url).load() for url in urls]

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
docs[0][0].page_content.strip()[:1000]

'Credit ratings News - Latest & Breaking News on credit ratings\n />\n\n\n\n\n\n\nSENSEX \xa0\xa082,515.14\n+\xa0123.42\n\nNIFTY \xa0\xa025,141.40\n+\xa037.15\n\nCRUDEOIL \xa0\xa05,649.00\n+\xa066.00\n\nGOLD \xa0\xa097,090.00\n+\xa0188.00\n\nSILVER \xa0\xa0105,775.00\n\xa0-974.00\n\n\n\n\n\n\nSENSEX \xa0\xa082,515.14\n+\xa0123.42\n\nNIFTY \xa0\xa025,141.40\n+\xa037.15\n\n\n\n\nNIFTY \xa0\xa025,141.40\n+\xa037.15\n\nCRUDEOIL \xa0\xa05,649.00\n+\xa066.00\n\n\n\n\nCRUDEOIL \xa0\xa05,649.00\n+\xa066.00\n\nGOLD \xa0\xa097,090.00\n+\xa0188.00\n\n\n\n\n\n\n\n\n\n \n\n\t\tCompanies\n\t\t\t\n\n\n\n\n\t\tMarkets\n\t\t\t\n\n\n\n\n\t\tPortfolio\n\t\t\t\n\n\n\n\n\t\tOpinion\n\t\t\t\n\n\n\n\n\t\tEconomy\n\t\t\t\n\n\n\n\n\t\tPremium\n\t\t\t\n\n\nFREE TRIAL ACCOUNT  SubscribeLOGIN MENU CLOSEePaperSTOCKSSTORIES\n\n\n\n\n            Companies\n            \n\n\n\n\n\n                            MSME\n                      \n\n\n\n\n            Markets\n            \n\n\n\n\n\n                           

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(docs_list)

In [6]:
doc_splits[0].page_content.strip()

'Credit ratings News - Latest & Breaking News on credit ratings\n />\n\n\n\n\n\n\nSENSEX \xa0\xa082,515.14\n+\xa0123.42\n\nNIFTY \xa0\xa025,141.40\n+\xa037.15\n\nCRUDEOIL \xa0\xa05,649.00\n+\xa066.00\n\nGOLD \xa0\xa097,090.00\n+\xa0188.00'

In [7]:
from langchain_aws import BedrockEmbeddings

embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0")

In [8]:
from langchain_core.vectorstores import InMemoryVectorStore
#vectorstore = InMemoryVectorStore(embeddings)
vectorstore = InMemoryVectorStore.from_documents(
    documents=doc_splits, embedding=embeddings
)

In [9]:
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['InMemoryVectorStore', 'BedrockEmbeddings'], vectorstore=<langchain_core.vectorstores.in_memory.InMemoryVectorStore object at 0x7f4aa88e7910>, search_kwargs={})

In [10]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_blog_posts",
    "Search and return information about Lilian Weng blog posts.",
)

In [11]:
retriever_tool.invoke({"query": "types of reward hacking"})

'Info-tech\n\t\t\t\t\n\n\n\n\t\t\t\t\tBL On Campus\n\t\t\t\t\n\n\n\n\t\t\t\t\tData Stories\n\t\t\t\t\n\n\n\n\t\t\t\t\tMarketing\n\t\t\t\t\n\n\n\n\t\t\t\t\tVideo\n\t\t\t\t\n\n\n\n\t\t\t\t\tPodcast\n\t\t\t\t\n\n\n\n\t\t\t\t\tPackage\n\nPolicy\n\nGold & Silver News\n\n    \n\n    Commodities News\n\n    \n\n    Personal Finance News\n\n    \n\n\n\t\t\tHolidays 2025\n\t\t\n\n\n    Bank Holidays 2025\n\n    \n\n    NSE Holiday 2025\n\n    \n\n    BSE Holiday 2025\n\n    \n\n\n\nPopular Sections\n\nBACK TO TOPSign into Unlock benefits!Access 10 free stories per monthAccess to comment on every storySign up/Manage to our newslettersGet notified by email for early preview to new features, discounts & offers✕Looks like you are already logged in from more than 3 devices!To continue logging in, remove at least one device from the below listLog outLog Out from all devicesTerms & conditions\xa0\xa0|\xa0\xa0Institutional Subscriber'

In [12]:
# Ensure your AWS credentials are configured

from langchain.chat_models import init_chat_model

llm = init_chat_model("us.anthropic.claude-3-5-sonnet-20240620-v1:0", model_provider="bedrock_converse")

In [13]:
from langgraph.graph import MessagesState
from langchain.chat_models import init_chat_model

#response_model = init_chat_model("openai:gpt-4.1", temperature=0)


response_model = init_chat_model("us.anthropic.claude-3-5-sonnet-20240620-v1:0", model_provider="bedrock_converse")


def generate_query_or_respond(state: MessagesState):
    """Call the model to generate a response based on the current state. Given
    the question, it will decide to retrieve using the retriever tool, or simply respond to the user.
    """
    response = (
        response_model
        .bind_tools([retriever_tool]).invoke(state["messages"])
    )
    return {"messages": [response]}

In [14]:
input = {"messages": [{"role": "user", "content": "hello!"}]}
generate_query_or_respond(input)["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hello! Welcome. How can I assist you today? I'm here to help with any questions you might have, particularly about Lilian Weng's blog posts. Is there a specific topic or information you're interested in?


In [15]:
input = {
    "messages": [
        {
            "role": "user",
            "content": "What is outlook of company DLF based on recent rating??",
        }
    ]
}
generate_query_or_respond(input)["messages"][-1].pretty_print()

================================== Ai Message ==================================

I apologize, but I don't have any specific information about recent ratings or the outlook for the company DLF. To provide accurate and up-to-date information, I would need to search for the latest data about DLF. 

However, the tools available to me are limited to searching for information about Lilian Weng's blog posts, which is not relevant to your question about DLF (which I assume is a company name).

If you're looking for financial information or stock ratings for DLF, I would recommend checking reliable financial websites, stock market analysis platforms, or contacting a financial advisor who has access to the most recent data about the company.

Is there perhaps a different topic you'd like to explore? I'd be happy to help with information that's within my knowledge base or that I can search for using the available tools.
